In [1]:
!uv add gradio google-generativeai python-dotenv nest-asyncio


  × No solution found when resolving dependencies for split                         
  │ (python_full_version >= '3.13'):
  ╰─▶ Because only the following versions of google-generativeai are
      available:
          google-generativeai==0.1.0
          google-generativeai==0.2.0
          google-generativeai==0.2.1
          google-generativeai==0.2.2
          google-generativeai==0.3.0
          google-generativeai==0.3.1
          google-generativeai==0.3.2
          google-generativeai==0.4.0
          google-generativeai==0.4.1
          google-generativeai==0.5.0
          google-generativeai==0.5.1
          google-generativeai==0.5.2
          google-generativeai==0.5.3
          google-generativeai==0.5.4
          google-generativeai==0.6.0
          google-generativeai==0.7.0
          google-generativeai==0.7.1
          google-generativeai==0.7.2
          google-generativeai==0.8.0
          google-generativeai==0.8.1
          google-generativeai==0.8.2
          googl

In [2]:
import openai
import gradio as gr
import os
from typing import List, Tuple
import json
from datetime import datetime
import nest_asyncio


In [4]:
nest_asyncio.apply()
load_dotenv(override=True)

NameError: name 'load_dotenv' is not defined

In [23]:
instructions_travel_manager= """
You act as a travel agent. A user tells you about their trip intention and your role is to find
what they are interested in doing and collect all the information needed to plan a trip and select
the right flight options and hotel. You keep asking questions and clarifications to the user
until you decide you have enough details to make a booking.
You are always polite and amicable to the user, and you are always transparent about the fact that you
are an AI Agent.
Once you have all the information required and you think the information can be passed to the operational team,
you thank the user, then record all the information in your memory so that it can be passed on the next agent
for research
"""

In [30]:
class TravelAgentBot:
    def __init__(self, api_key: str):
        """Initialize the travel agent bot with OpenAI API"""
        self.client = openai.OpenAI(api_key=api_key)
        self.conversation_history = []
        
        # Travel agent system prompt
        self.system_prompt = instructions_travel_manager

    def generate_response(self, user_message: str) -> str:
        """Generate a response using OpenAI"""
        try:
            # Build messages for the conversation
            messages = [{"role": "system", "content": self.system_prompt}]
            
            # Add conversation history for context
            for exchange in self.conversation_history[-5:]:  # Keep last 5 exchanges
                messages.append({"role": "user", "content": exchange['user']})
                messages.append({"role": "assistant", "content": exchange['assistant']})
            
            # Add current user message
            messages.append({"role": "user", "content": user_message})
            
            # Generate response
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",  # or "gpt-4" if you have access
                messages=messages,
                max_tokens=1000,
                temperature=0.7
            )
            
            assistant_response = response.choices[0].message.content
            
            # Store in conversation history
            self.conversation_history.append({
                "user": user_message,
                "assistant": assistant_response,
                "timestamp": datetime.now().isoformat()
            })
            
            return assistant_response
            
        except Exception as e:
            return f"I'm sorry, I encountered an error: {str(e)}. Please try again."

    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        return "Conversation history cleared. How can I help you plan your next adventure?"


In [ ]:
API_KEY = os.getenv("GEMINI_API_KEY")
bot = TravelAgentBot(API_KEY)
print("✅ Travel Agent Bot initialized successfully!")
print("🌍 Ready to help you plan your next adventure!")

In [32]:
def chat_with_bot(message: str, history: List[Tuple[str, str]]) -> Tuple[List[Tuple[str, str]], str]:
    """Handle chat interaction"""
    if not message.strip():
        return history, ""
    
    # Generate response
    response = bot.generate_response(message)
    
    # Update chat history
    history.append((message, response))
    
    return history, ""

def clear_chat():
    """Clear chat history"""
    bot.clear_history()
    return [], ""

def get_conversation_info():
    """Get conversation statistics"""
    return bot.get_conversation_summary()

In [33]:
def create_travel_bot_interface():
    with gr.Blocks(
        title="🌍 AI Travel Agent",
        theme=gr.themes.Soft(),
    ) as demo:
        
        # Header
        gr.HTML("""
        <div style="text-align: center; padding: 20px; background: linear-gradient(90deg, #4CAF50, #45a049); color: white; border-radius: 10px; margin-bottom: 20px;">
            <h1>🌍 AI Travel Agent Bot</h1>
            <p>Your personal travel planning assistant powered by Gemini 1.5 Flash</p>
        </div>
        """)
        
        # Main chat interface
        with gr.Row():
            with gr.Column(scale=4):
                chatbot = gr.Chatbot(
                    label="💬 Chat with your Travel Agent",
                    height=500,
                    show_label=True,
                    container=True,
                    bubble_full_width=False,
                    avatar_images=("🧳", "🤖")
                )
                
                with gr.Row():
                    msg_input = gr.Textbox(
                        label="Your message",
                        placeholder="Ask me about destinations, hotels, activities, or travel tips...",
                        lines=2,
                        show_label=False,
                        container=False
                    )
                    
                with gr.Row():
                    send_btn = gr.Button("Send ✈️", variant="primary", size="sm")
                    clear_btn = gr.Button("Clear Chat 🗑️", variant="secondary", size="sm")
            
            # Sidebar with info and controls
            with gr.Column(scale=1):
                gr.HTML("<h3>🔧 Controls</h3>")
                
                info_btn = gr.Button("📊 Conversation Info", size="sm")
                info_output = gr.Textbox(
                    label="Conversation Summary",
                    lines=6,
                    max_lines=10,
                    interactive=False
                )
                
                gr.HTML("""
                <div style="margin-top: 20px; padding: 10px; background-color: #e8f5e8; border-radius: 5px;">
                    <h4>💡 Quick Tips:</h4>
                    <ul style="font-size: 12px;">
                        <li>Be specific about dates and budget</li>
                        <li>Mention travel preferences</li>
                        <li>Ask for detailed itineraries</li>
                        <li>Request local recommendations</li>
                    </ul>
                </div>
                """)
        
        # Example questions section
        with gr.Row():
            gr.HTML("""
            <div style="margin-top: 20px; padding: 15px; background-color: #f0f8ff; border-radius: 10px;">
                <h3>💡 Example Questions to Get Started:</h3>
                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; margin-top: 10px;">
                    <div>
                        <strong>🏝️ Destination Planning:</strong>
                        <ul>
                            <li>"Plan a 7-day trip to Japan in spring"</li>
                            <li>"Best budget destinations in Europe for couples"</li>
                            <li>"Family-friendly activities in Bali"</li>
                        </ul>
                    </div>
                    <div>
                        <strong>🍽️ Food & Culture:</strong>
                        <ul>
                            <li>"Food tour itinerary for Thailand"</li>
                            <li>"Cultural etiquette tips for visiting Morocco"</li>
                            <li>"Best local markets in Istanbul"</li>
                        </ul>
                    </div>
                </div>
            </div>
            """)
        
        # Event handlers
        send_btn.click(
            chat_with_bot,
            inputs=[msg_input, chatbot],
            outputs=[chatbot, msg_input]
        )
        
        msg_input.submit(
            chat_with_bot,
            inputs=[msg_input, chatbot],
            outputs=[chatbot, msg_input]
        )
        
        clear_btn.click(
            clear_chat,
            outputs=[chatbot, msg_input]
        )
        
        info_btn.click(
            get_conversation_info,
            outputs=[info_output]
        )
    
    return demo


In [ ]:
# Cell 7: Launch the interface inline in notebook
print("🚀 Starting Travel Agent Bot interface...")
print("📱 The interface will appear below in this notebook")

# Create the demo
demo = create_travel_bot_interface()

# Enable queue for better notebook performance
demo.queue()

# Launch inline within the notebook
demo.launch(
    debug=True,
    share=False,
    inbrowser=False,  # Keep in notebook
    inline=True,      # Display inline
    height=600,       # Set height
    show_error=True,
    quiet=False
)